![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Question Answering and Summarization with [T5](https://arxiv.org/pdf/1910.10683.pdf)


Google's T5 is a Sequence to Sequence model that was trained on over 15 different NLP datasets with various problem types, raning from Text Summarization, Question Answering, Translation to various semantical deduction tasks, which enriches T5's ability to map token sequences to semantic vectors which contain more meaning, which T5 leverages to generalize across various tasks and even to never before trained tasks.

On top of this, T5 is trained on the standard Word prediction task, which most transformer based models like BERT, GPT, ELMO have been trained on. This gives T5 general knowledge of real world concepts to additionally enhance its understanding.

With T5 you can answer **general knowledge based questions given no context** and in addition answer **questions on text databases**.      
These questions can be asked in natural human.


## What is a `open book question`? 
You can imagine an `open book` question similar to an examen where you are allowed to bring in text documents or cheat sheets that help you answer questions in an examen. Kinda like bringing a history book to an history examen. 

In `T5's` terms, this means the model is given a `question` and an **additional piece of textual information** or so called `context`.

This enables the `T5` model to answer questions on textual datasets like `medical records`,`newsarticles` , `wiki-databases` , `stories` and `movie scripts` , `product descriptions`, 'legal documents' and many more.



## What is a `closed book question`? 
A `closed book question` is the exact opposite of a `open book question`. In an examen scenario, you are only allowed to use what you have memorized in your brain and nothing else.      
In `T5's` terms this means that T5 can only use it's stored weights to answer a `question` and is given **no aditional context**.        
`T5` was pre-trained on the [C4 dataset](https://commoncrawl.org/) which contains **petabytes  of web crawling data**  collected over the last 8 years, including Wikipedia in every language.


This gives `T5` the broad knowledge of the internet stored in it's weights to answer various `closed book questions` 



<!-- [T5]() -->
![T5 GIF](https://1.bp.blogspot.com/-o4oiOExxq1s/Xk26XPC3haI/AAAAAAAAFU8/NBlvOWB84L0PTYy9TzZBaLf6fwPGJTR0QCLcBGAsYHQ/s1600/image3.gif)


## Start Spark Session

In [5]:
import json
import os

import sparknlp
from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

print("Spark NLP Version :", sparknlp.version())

spark = start_spark()
spark.sparkContext.setLogLevel("ERROR")

spark

Spark NLP Version : 4.2.4
Spark Session already created, some configs may not take.


# Download T5 Model and Create Spark NLP Pipeline

In [6]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") 

# Can take in document or sentence columns
t5 = T5Transformer.pretrained(name='t5_base',lang='en')\
    .setInputCols('document')\
    .setOutputCol("T5")\
    .setMaxOutputLength(400)

t5_base download started this may take some time.
Approximate size to download 451.8 MB
[ | ]t5_base download started this may take some time.
Approximate size to download 451.8 MB
[ | ]Download done! Loading the resource.
[OK!]


# Answering Questions 

## Set the Task to `question`


In [7]:
# Set the task for questions on T5. Depending to what this is currently set, we get different behaivour
t5.setTask('question')

T5TRANSFORMER_8078c2d39352

## Answer **Closed Book Questions**  
Closed book means that no additional context is given and the model must answer the question with the knowledge stored in it's weights

In [8]:
# Build pipeline with T5
pipe_components = [documentAssembler,t5]
pipeline = Pipeline().setStages( pipe_components)

# define Data
data = [["Who is president of Nigeria? "],
        ["What is the most common language in India? "],
        ["What is the capital of Germany? "]]
df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

+-------------------------------------------+------------------+
|text                                       |result            |
+-------------------------------------------+------------------+
|Who is president of Nigeria?               |[Muhammadu Buhari]|
|What is the most common language in India? |[Hindi]           |
|What is the capital of Germany?            |[Berlin]          |
+-------------------------------------------+------------------+



## Answer **Open Book Questions** 
These are questions where we give the model some additional context, that is used to answer the question

In [9]:
from pyspark.sql import SparkSession

context    = 'context: Peters last week was terrible! He had an accident and broke his leg while skiing!'
question1  = 'question: Why was peters week so bad? ' #
question2  = 'question: How did peter broke his leg? ' 
question3  = 'question: How did peter broke his leg? '
 
data = [[question1+context],[question2+context],[question3+context],]
df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+
|text                                                                                                                             |result                                             |
+---------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+
|question: Why was peters week so bad? context: Peters last week was terrible! He had an accident and broke his leg while skiing! |[He had an accident and broke his leg while skiing]|
|question: How did peter broke his leg? context: Peters last week was terrible! He had an accident and broke his leg while skiing!|[skiing]                                           |
|question: How did peter broke his leg? context: Peters last week was terrible! 

In [10]:
# Ask T5 questions in the context of a News Article
question1 = 'question: Who is Jack ma? '
question2 = 'question: Who is founder of Alibaba Group? '
question3 = 'question: When did Jack Ma re-appear? '
question4 = 'question: How did Alibaba stocks react? '
question5 = 'question: Whom did Jack Ma meet? '
question6 = 'question: Who did Jack Ma hide from? '


# from https://www.bbc.com/news/business-55728338 
news_article_context = """ context:
Alibaba Group founder Jack Ma has made his first appearance since Chinese regulators cracked down on his business empire.
His absence had fuelled speculation over his whereabouts amid increasing official scrutiny of his businesses.
The billionaire met 100 rural teachers in China via a video meeting on Wednesday, according to local government media.
Alibaba shares surged 5% on Hong Kong's stock exchange on the news.
"""

data = [
             [question1+ news_article_context],
             [question2+ news_article_context],
             [question3+ news_article_context],
             [question4+ news_article_context],
             [question5+ news_article_context],
             [question6+ news_article_context]]


df=spark.createDataFrame(data).toDF('text')

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['t5.result']).show(truncate=False)

+-----------------------+
|result                 |
+-----------------------+
|[Alibaba Group founder]|
|[Jack Ma]              |
|[Wednesday]            |
|[surged 5%]            |
|[100 rural teachers]   |
|[Chinese regulators]   |
+-----------------------+



# Summarize documents

In [11]:
# Set the task for questions on T5
t5.setTask('summarize')

T5TRANSFORMER_8078c2d39352

In [12]:
# https://www.reuters.com/article/instant-article/idCAKBN2AA2WF
text = """(Reuters) - Mastercard Inc said on Wednesday it was planning to offer support for some cryptocurrencies on its network this year, joining a string of big-ticket firms that have pledged similar support.

The credit-card giant’s announcement comes days after Elon Musk’s Tesla Inc revealed it had purchased $1.5 billion of bitcoin and would soon accept it as a form of payment.

Asset manager BlackRock Inc and payments companies Square and PayPal have also recently backed cryptocurrencies.

Mastercard already offers customers cards that allow people to transact using their cryptocurrencies, although without going through its network.

"Doing this work will create a lot more possibilities for shoppers and merchants, allowing them to transact in an entirely new form of payment. This change may open merchants up to new customers who are already flocking to digital assets," Mastercard said. (mstr.cd/3tLaPZM)

Mastercard specified that not all cryptocurrencies will be supported on its network, adding that many of the hundreds of digital assets in circulation still need to tighten their compliance measures.

Many cryptocurrencies have struggled to win the trust of mainstream investors and the general public due to their speculative nature and potential for money laundering.
"""
data = [[text]]
df=spark.createDataFrame(data).toDF('text')
#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['t5.result']).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
v = annotated_df.take(1)
print(f"Original Length {len(v[0].text)}   Summarized Length : {len(v[0].T5[0].result)} ")


Original Length 1284   Summarized Length : 352 


In [14]:
# Full summarized text
v[0].T5[0].result

'mastercard said on Wednesday it was planning to offer support for some cryptocurrencies on its network this year . the credit-card giant’s announcement comes days after Elon Musk’s Tesla Inc revealed it had purchased $1.5 billion of bitcoin . asset manager blackrock and payments companies Square and PayPal have also recently backed cryptocurrencies .'

# Multi Problem T5 model for Summarization and more
The main T5 model was trained for over 20 tasks from the SQUAD/GLUE/SUPERGLUE datasets. See [this notebook](https://github.com/JohnSnowLabs/nlu/blob/master/examples/webinars_conferences_etc/multi_lingual_webinar/7_T5_SQUAD_GLUE_SUPER_GLUE_TASKS.ipynb) for a demo of all tasks 


# Overview of every task available with T5
[The T5 model](https://arxiv.org/pdf/1910.10683.pdf) is trained on various datasets for 17 different tasks which fall into 8 categories.



1. Text summarization
2. Question answering
3. Translation
4. Sentiment analysis
5. Natural Language inference
6. Coreference resolution
7. Sentence Completion
8. Word sense disambiguation

### Every T5 Task with explanation:
|Task Name | Explanation | 
|----------|--------------|
|[1.CoLA](https://nyu-mll.github.io/CoLA/)                   | Classify if a sentence is gramaticaly correct|
|[2.RTE](https://dl.acm.org/doi/10.1007/11736790_9)                    | Classify whether if a statement can be deducted from a sentence|
|[3.MNLI](https://arxiv.org/abs/1704.05426)                   | Classify for a hypothesis and premise whether they contradict or contradict each other or neither of both (3 class).|
|[4.MRPC](https://www.aclweb.org/anthology/I05-5002.pdf)                   | Classify whether a pair of sentences is a re-phrasing of each other (semantically equivalent)|
|[5.QNLI](https://arxiv.org/pdf/1804.07461.pdf)                   | Classify whether the answer to a question can be deducted from an answer candidate.|
|[6.QQP](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)                    | Classify whether a pair of questions is a re-phrasing of each other (semantically equivalent)|
|[7.SST2](https://www.aclweb.org/anthology/D13-1170.pdf)                   | Classify the sentiment of a sentence as positive or negative|
|[8.STSB](https://www.aclweb.org/anthology/S17-2001/)                   | Classify the sentiment of a sentence on a scale from 1 to 5 (21 Sentiment classes)|
|[9.CB](https://ojs.ub.uni-konstanz.de/sub/index.php/sub/article/view/601)                     | Classify for a premise and a hypothesis whether they contradict each other or not (binary).|
|[10.COPA](https://www.aaai.org/ocs/index.php/SSS/SSS11/paper/view/2418/0)                   | Classify for a question, premise, and 2 choices which choice the correct choice is (binary).|
|[11.MultiRc](https://www.aclweb.org/anthology/N18-1023.pdf)                | Classify for a question, a paragraph of text, and an answer candidate, if the answer is correct (binary),|
|[12.WiC](https://arxiv.org/abs/1808.09121)                    | Classify for a pair of sentences and a disambigous word if the word has the same meaning in both sentences.|
|[13.WSC/DPR](https://www.aaai.org/ocs/index.php/KR/KR12/paper/view/4492/0)       | Predict for an ambiguous pronoun in a sentence what it is referring to.  |
|[14.Summarization](https://arxiv.org/abs/1506.03340)          | Summarize text into a shorter representation.|
|[15.SQuAD](https://arxiv.org/abs/1606.05250)                  | Answer a question for a given context.|
|[16.WMT1.](https://arxiv.org/abs/1706.03762)                  | Translate English to German|
|[17.WMT2.](https://arxiv.org/abs/1706.03762)                   | Translate English to French|
|[18.WMT3.](https://arxiv.org/abs/1706.03762)                   | Translate English to Romanian|

